# Part 2: Time Series Features & Tree-Based Models

**Objective:** Extract basic time-series features from heart rate data, train Random Forest and XGBoost models, and compare their performance.

## 1. Setup

Import necessary libraries.

In [34]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer

## 2. Data Loading

Load the dataset.

In [35]:
def load_data(file_path: str = "data/synthetic_health_data.csv") -> pd.DataFrame:
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data with timestamp parsed as datetime
    """
    # YOUR CODE HERE
    # Load the CSV file using pandas
    # Make sure to parse the timestamp column as datetime
    try:
        data = pd.read_csv(file_path)
        print(f"Data loaded successfully from {file_path}")
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        assert data['timestamp'].dtype == 'datetime64[ns]', "Timestamp column is not in datetime format"
        return data
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None

df = load_data()

Data loaded successfully from data/synthetic_health_data.csv


## 3. Feature Engineering

Implement `extract_rolling_features` to calculate rolling mean and standard deviation for the `heart_rate`.

In [36]:
def extract_rolling_features(df: pd.DataFrame, window_size_seconds : float = 60) -> pd.DataFrame:
    """
    Calculate rolling mean and standard deviation for heart rate.
    
    Args:
        df: DataFrame with timestamp and heart_rate columns
        window_size_seconds: Size of the rolling window in seconds
        
    Returns:
        DataFrame with added hr_rolling_mean and hr_rolling_std columns
    """
    # YOUR CODE HERE
    # 1. Sort data by timestamp
    df_sorted = df.sort_values('timestamp')
    # 2. Set timestamp as index (this allows time-based operations)
    df_indexed = df_sorted.set_index('timestamp')
    # 3. Calculate rolling mean and standard deviation
    #    - First, create a rolling window object based on time:
    rolling_window = df_indexed['heart_rate'].rolling(window = pd.Timedelta(seconds=window_size_seconds))
    #    - Then calculate statistics on this window:
    hr_mean = rolling_window.mean()
    hr_std = rolling_window.std()
    # 4. Add the new columns back to the dataframe
    df_indexed['hr_rolling_mean'] = hr_mean
    df_indexed['hr_rolling_std'] = hr_std
    # 5. Reset index to bring timestamp back as a column
    df_with_feature = df_indexed.reset_index()
    # 6. Handle any NaN values (rolling calculations create NaNs at the beginning)
    #    - You can use fillna, dropna, or other methods depending on your strategy
    df_with_feature = df_with_feature.ffill().bfill()
    assert df_with_feature.isna().sum().sum() == 0, "still have " + str(df_with_feature.isna().sum().sum()) + " NaN values"
    # Placeholder return - replace with your implementation
    return df_with_feature.copy()
df = extract_rolling_features(df)

## 4. Data Preparation

Implement `prepare_data_part2` using the newly engineered features.

In [37]:
def prepare_data_part2(df_with_features: pd.DataFrame, test_size=0.2, random_state=42) -> tuple:
    """
    Prepare data for modeling with time-series features.
    
    Args:
        df_with_features: DataFrame with original and rolling features
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    # YOUR CODE HERE
    # 1. Select relevant features including the rolling features
    X = df_with_features[['heart_rate', 'hr_rolling_mean', 'hr_rolling_std']].copy()
    # 2. Select target variable (disease_outcome)
    Y = df_with_features['disease_outcome'].copy()
    # 3. Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)
    # 4. Handle missing values
    assert X_train.isnull().sum().sum() == 0, "Missing values in training data"
    assert X_test.isnull().sum().sum() == 0, "Missing values in testing data"
    
    # Placeholder return - replace with your implementation
    return (X_train, X_test, y_train, y_test)

## 5. Random Forest Model

Implement `train_random_forest`.

In [38]:
def train_random_forest(X_train, y_train, n_estimators=100, max_depth=10, random_state=42) -> RandomForestClassifier:
    """
    Train a Random Forest classifier.
    
    Args:
        X_train: Training features
        y_train: Training target
        n_estimators: Number of trees in the forest
        max_depth: Maximum depth of the trees
        random_state: Random seed for reproducibility
        
    Returns:
        Trained Random Forest model
    """
    # YOUR CODE HERE
    # Initialize and train a RandomForestClassifier
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)
    
    return model  # Replace with actual implementation

## 6. XGBoost Model

Implement `train_xgboost`.

In [39]:
def train_xgboost(X_train, y_train, n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42) -> xgb.XGBClassifier:
    """
    Train an XGBoost classifier.
    
    Args:
        X_train: Training features
        y_train: Training target
        n_estimators: Number of boosting rounds
        learning_rate: Boosting learning rate
        max_depth: Maximum depth of a tree
        random_state: Random seed for reproducibility
        
    Returns:
        Trained XGBoost model
    """
    # YOUR CODE HERE
    # Initialize and train an XGBClassifier
    model = xgb.XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)
    
    return model  # Replace with actual implementation

## 7. Model Comparison

Calculate and compare AUC scores for both models.

In [40]:
# YOUR CODE HERE
# 1. Generate probability predictions for both models
# 2. Calculate AUC scores
# 3. Compare the performance
    # rf_probs = rf_model.predict_proba(X_test)[:, 1]
    # xgb_probs = xgb_model.predict_proba(X_test)[:, 1]
    
    # rf_auc = roc_auc_score(y_test, rf_probs)
    # xgb_auc = roc_auc_score(y_test, xgb_probs)

## 8. Save Results

Save the AUC scores to a text file.

In [41]:
# YOUR CODE HERE
# 1. Create 'results' directory if it doesn't exist
# 2. Format AUC scores as strings
# 3. Write scores to 'results/results_part2.txt'

## 9. Main Execution

Run the complete workflow.

In [48]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Extract rolling features
    window_size = 300  # 5 minutes in seconds
    df_with_features = extract_rolling_features(df, window_size)
    
    # 3. Prepare data
    X_train, X_test, y_train, y_test = prepare_data_part2(df_with_features)
    
    # 4. Train models
    rf_model = train_random_forest(X_train, y_train)
    xgb_model = train_xgboost(X_train, y_train)
    
    # 5. Calculate AUC scores
    rf_probs = rf_model.predict_proba(X_test)[:, 1]
    xgb_probs = xgb_model.predict_proba(X_test)[:, 1]
    
    rf_auc = roc_auc_score(y_test, rf_probs)
    xgb_auc = roc_auc_score(y_test, xgb_probs)
    
    print(f"Random Forest AUC: {rf_auc:.4f}")
    print(f"XGBoost AUC: {xgb_auc:.4f}")
    print('Random Forest'  if rf_auc > xgb_auc else 'XGBoost' + ' is better')
    
    # 6. Save results
    # (Your code for saving results)

Data loaded successfully from data/synthetic_health_data.csv
Random Forest AUC: 0.7751
XGBoost AUC: 0.7759
XGBoost is better
Random Forest AUC: 0.7751
XGBoost AUC: 0.7759
XGBoost is better
